<a href="https://colab.research.google.com/github/zzghost2000/lai/blob/master/ex1_CNN_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN + Data Augmentation 練習

In [0]:
# Check GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# import package
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## Cifar10 資料讀入及前處理

In [0]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[:10000,:,:,:]
y_train = y_train[:10000,]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

164864000/170498071 [============================>.] - ETA: 1s

In [0]:
# image preprocessing
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255  # rescaling
x_test = x_test/255  # rescaling

# show image

fig = plt.figure()
plt.subplot(121)
plt.grid(False)
plt.imshow(x_train[0])

plt.subplot(122)
plt.grid(False)
plt.imshow(x_train[1])

plt.show()

In [0]:
# Convert class vectors to binary class matrices.
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)

In [0]:
## result's dictionary
model_result = dict()
model_result['acc'] = dict()
model_result['loss'] = dict()
model_result['val_acc'] = dict()
model_result['val_loss'] = dict()

## 模型定義

In [0]:
#@title { vertical-output: true }
# define model structure.
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

In [0]:
# initiate optimizer

learning_rate = 0.001
optimizer = keras.optimizers.Adam(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Data Augmentation

In [0]:
#@title Hyperparameter Setting { vertical-output: true }
data_augmentation = True #@param {type:"boolean"}
batch_size = 32 #@param {type:"integer"}
epochs = 15 #@param {type:"integer"}


In [0]:
#@title Data Augmentation Parameter Setting { vertical-output: true }
if data_augmentation:
    featurewise_center = False #@param {type:"boolean"}
    samplewise_center = False #@param {type:"boolean"}
    featurewise_std_normalization = False #@param {type:"boolean"}
    samplewise_std_normalization = False #@param {type:"boolean"}
    rotation_range = 0 #@param {type:"slider", min:0, max:180, step:1}
    width_shift_range = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
    height_shift_range = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
    horizontal_flip = True #@param {type:"boolean"}
    vertical_flip = False #@param {type:"boolean"}
    fill_mode = 'constant' #@param ["nearest", "constant", "reflect", "wrap"] {type:"string"}
    cval = 0 #@param {type:"slider", min:0, max:255, step:1}

## 開始訓練模型 (Data augmentation 參數設置)

In [0]:
#@title { vertical-output: true }
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    # https://keras.io/preprocessing/image/
    datagen = ImageDataGenerator(
        featurewise_center=featurewise_center,  # 為使數據集去中心化(使均值為0)
        samplewise_center=samplewise_center,  # 使輸入數據的每個樣本均值為0
        featurewise_std_normalization=featurewise_std_normalization,  # 將輸入除以數據集的標準差以完成標準化
        samplewise_std_normalization=samplewise_std_normalization,  # 將輸入的每個樣本除以其自身的標準差
        rotation_range=rotation_range, # 隨機旋轉的度數範圍。 (degrees, 0 to 180)
        width_shift_range=width_shift_range,  # 隨機水平移動的範圍。 (總高度的比例)
        height_shift_range=height_shift_range,  # 隨機垂直移動的範圍。 (總高度的比例)
        horizontal_flip=horizontal_flip,  # 隨機水平翻轉。
        vertical_flip=vertical_flip,  # 隨機垂直翻轉
        fill_mode=fill_mode,  # 補全缺失圖片值
        cval=cval)  ## 補浮點數或整數用於邊界之外的點的值 (當fill mode = constant時需要用到)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size +1,
                        epochs=epochs,
                        validation_data=(x_test, y_test))
    
    
# data Augmentation後的圖片
print('After Data augmentation ：')
datagenerator = datagen.flow(x_train[0:2],batch_size=2)

img_after_augmentation = datagenerator.next()

# show the image

fig = plt.figure()
plt.subplot(121)
plt.grid(False)
plt.imshow(img_after_augmentation[0])

plt.subplot(122)
plt.grid(False)
plt.imshow(img_after_augmentation[1])

plt.show()

## 訓練結果視覺化

In [0]:
#@title Save Learning Result { vertical-output: true }
experiment = "data augmentation"

model_result['acc'][experiment] = model.model.history.history.get('acc')
model_result['val_acc'][experiment] = model.model.history.history.get('val_acc')
model_result['loss'][experiment] = model.model.history.history.get('loss')
model_result['val_loss'][experiment] = model.model.history.history.get('val_loss')

In [0]:
#@title Model Result { vertical-output: true }
import matplotlib.pyplot as plt

# training loss
for i in model_result['loss'].keys():
    plt.plot(np.arange(len(model_result['loss'][i])), model_result['loss'][i], label = i)

plt.legend(loc='best')
plt.title('loss')
plt.show()

# training acc
for i in model_result['acc'].keys():
    plt.plot(np.arange(len(model_result['acc'][i])), model_result['acc'][i], label = i)

plt.legend(loc='best')
plt.title('acc')
plt.show()

# valid loss
for i in model_result['val_loss'].keys():
    plt.plot(np.arange(len(model_result['val_loss'][i])), model_result['val_loss'][i], label = i)

plt.legend(loc='best')
plt.title('val_loss')
plt.show()


# valid acc
for i in model_result['val_acc'].keys():
    plt.plot(np.arange(len(model_result['val_acc'][i])), model_result['val_acc'][i], label = i)

plt.legend(loc='best')
plt.title('val_acc')
plt.show()